In [1]:
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from model import *
from process import *

In [2]:
data='cora'
adj, features, labels,idx_train,idx_val,idx_test = load_citation(data)
#splitstr = 'splits/'+data+'_split_0.6_0.2_'+str(0)+'.npz'
#adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data(data,splitstr)
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)
print(idx_val.shape)
print(idx_test.shape)

torch.Size([2708, 2708])
torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([140])
torch.Size([500])
torch.Size([1000])


C:\Users\ayday\Desktop\Master_Thesis\experiment\utils.py:87: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


In [3]:
cudaid = "cuda"
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device).coalesce()

In [4]:
n=features.shape[0]
feature_size=features.shape[1]
num_classes = len(torch.unique(labels))
hidden_dim=1024
dropout=0.5
model=my_GCN(n,feature_size,hidden_dim,num_classes,dropout)
model=model.to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)

epochs=500
patience=10
test=False

In [5]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    #model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
    
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()
    if(epoch+1)%1 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.2f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.2f}'.format(acc_val*100))
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        #torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == patience:
        break

#if test:
#    acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Val","acc.:{:.1f}".format(acc*100))

Epoch:0001 train loss:1.988 acc:12.14 | val loss:1.741 acc:26.80
Epoch:0002 train loss:1.187 acc:64.29 | val loss:1.115 acc:76.00
Epoch:0003 train loss:0.495 acc:99.29 | val loss:0.973 acc:71.20
Epoch:0004 train loss:0.247 acc:97.14 | val loss:0.853 acc:74.60
Epoch:0005 train loss:0.114 acc:98.57 | val loss:0.766 acc:76.80
Epoch:0006 train loss:0.045 acc:100.00 | val loss:0.761 acc:76.00
Epoch:0007 train loss:0.024 acc:100.00 | val loss:0.786 acc:75.80
Epoch:0008 train loss:0.017 acc:100.00 | val loss:0.806 acc:75.20
Epoch:0009 train loss:0.011 acc:100.00 | val loss:0.820 acc:75.80
Epoch:0010 train loss:0.008 acc:100.00 | val loss:0.832 acc:75.20
Epoch:0011 train loss:0.006 acc:100.00 | val loss:0.846 acc:75.20
Epoch:0012 train loss:0.004 acc:100.00 | val loss:0.862 acc:75.00
Epoch:0013 train loss:0.003 acc:100.00 | val loss:0.879 acc:74.40
Epoch:0014 train loss:0.003 acc:100.00 | val loss:0.895 acc:74.60
Epoch:0015 train loss:0.004 acc:100.00 | val loss:0.900 acc:74.80
Epoch:0016 trai

In [6]:
# [2708, 16]
# [2708, 7]

In [7]:

print(adj.to_dense().shape)
        print(support.shape)
        print(output.shape)


torch.Size([2708, 2708]) adj
torch.Size([2708, 16])  support= input x weight
torch.Size([2708, 2724])

IndentationError: unexpected indent (1557039151.py, line 2)